Setup

In [4]:
from pydrake.all import (
    DiagramBuilder, StartMeshcat, Simulator
)
from manipulation.station import LoadScenario, MakeHardwareStation, AddPointClouds
from manipulation.utils import RenderDiagram
from setup_simulation import get_scenario

In [2]:
# Setup meshcat for visualization
meshcat = StartMeshcat()
print('Click the link above to open Meshcat in your browser!')

INFO:drake:Meshcat listening for connections at http://localhost:7000


Click the link above to open Meshcat in your browser!


In [9]:
# Get scenario
scenario_string = get_scenario()

# Load the scenario and build the simulation station
scenario = LoadScenario(data=scenario_string)
station = MakeHardwareStation(scenario, meshcat=meshcat)

# Build a Drake Diagram containing the station
builder = DiagramBuilder()
builder.AddSystem(station)

# Setup getting point clouds from the cameras
to_point_cloud = AddPointClouds(
    scenario=scenario, station=station, builder=builder, meshcat=meshcat
)
builder.ExportOutput(to_point_cloud["camera0"].get_output_port(), "camera0_point_cloud")
builder.ExportOutput(to_point_cloud["camera1"].get_output_port(), "camera1_point_cloud")
builder.ExportOutput(to_point_cloud["camera2"].get_output_port(), "camera2_point_cloud")

# Build diagram
diagram = builder.Build()

ERROR: In vtkPNGReader.cxx, line 133
vtkPNGReader (0x1efbb1a0): Unknown file type! Not a PNG file!

ERROR: In vtkPNGReader.cxx, line 275
vtkPNGReader (0x1efbb1a0): Invalid file header: not a PNG file

ERROR: In vtkExecutive.cxx, line 729
vtkCompositeDataPipeline (0x1efba2e0): Algorithm vtkPNGReader (0x1efbb1a0) returned failure for request: vtkInformation (0x1efbe270)
  Debug: Off
  Modified Time: 204324
  Reference Count: 1
  Registered Events: (none)
  Request: REQUEST_INFORMATION
  ALGORITHM_AFTER_FORWARD: 1
  FORWARD_DIRECTION: 0



ERROR: In vtkStreamingDemandDrivenPipeline.cxx, line 852
vtkCompositeDataPipeline (0x1efba2e0): No whole extent has been set in the information for output port 0 on algorithm vtkPNGReader (0x1efbb1a0).



In [ ]:
RenderDiagram(diagram, max_depth=1)

In [ ]:
# Create and run a simulator
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(0.1)